In [1]:
import numpy as np
import paths

import pygraph

from pygraph.utils.graphfiles import loadDataset


In [2]:
import networkx as nx
import numpy as np
import matplotlib.pyplot  as plt

# We load a ds dataset
# load it from https://brunl01.users.greyc.fr/CHEMISTRY/Acyclic.tar.gz
dataset, y = loadDataset("/home/bgauzere/work/Datasets/Acyclic/dataset_bps.ds")

In [3]:
#Compute graph edit distances

from tqdm import tqdm
from pygraph.c_ext.lsape_binders import  lsap_solverHG
from pygraph.ged.costfunctions import ConstantCostFunction
from pygraph.ged.GED import ged
import time

cf = ConstantCostFunction(1,3,1,3)
N=len(dataset)

methods=['Riesen + LSAP', 'Neigh + LSAP', 'Riesen + LSAPE', 'Neigh + LSAPE']
ged_distances = [ np.zeros((N,N)), np.zeros((N,N)), np.zeros((N,N)), np.zeros((N,N))]

times = list()
start = time.clock()
for i in tqdm(range(0,N)):
    for j in range(0,N):
        ged_distances[0][i,j] = ged(dataset[i],dataset[j],cf=cf, method='Riesen')[0]
times.append(time.clock() - start)


start = time.clock()
for i in tqdm(range(0,N)):
    for j in range(0,N):
        ged_distances[1][i,j] = ged(dataset[i],dataset[j],cf=cf, method='Neighboorhood')[0]

times.append(time.clock() - start)

start = time.clock()
for i in tqdm(range(0,N)):
    for j in range(0,N):
        ged_distances[2][i,j] = ged(dataset[i],dataset[j],cf=cf, method='Riesen',solver=lsap_solverHG)[0]
times.append(time.clock() - start)

start = time.clock()
for i in tqdm(range(0,N)):
    for j in range(0,N):
        ged_distances[3][i,j] = ged(dataset[i],dataset[j],cf=cf, method='Neighboorhood',solver=lsap_solverHG)[0]
times.append(time.clock() - start)

100%|██████████| 183/183 [05:50<00:00,  1.91s/it]


In [5]:
print(" method \t mean \t mean \t time")
data = list()
for i in range(0,len(ged_distances)):
    ged_ = np.minimum(ged_distances[i],ged_distances[i].transpose())
    print(" {} \t {} \t {} \t {} ".format(methods[i], np.mean(ged_distances[i]),np.mean(ged_), times[i]))


 method 	 mean 	 mean 	 time
 Riesen + LSAP 	 37.79903849025053 	 35.31207262086058 	 463.300405 
 Neigh + LSAP 	 36.2281047508137 	 33.85869987159963 	 521.7821730000001 
 Riesen + LSAPE 	 35.95508973095643 	 34.10092866314312 	 319.83455500000014 
 Neigh + LSAPE 	 34.5005822807489 	 32.5735614679447 	 350.48029599999995 
